In [1]:
# import glob
# import pathlib

# from langchain.document_loaders import PyPDFDirectoryLoader
# from langchain.document_loaders import PyPDFLoader 
# from langchain.embeddings import OpenAIEmbeddings 
# from langchain.vectorstores import Chroma 
# from langchain.chains import ConversationalRetrievalChain
# from langchain.memory import ConversationBufferMemory
# from langchain.llms import OpenAI

# from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.indexes import VectorstoreIndexCreator

# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# import tiktoken
# import pinecone

from utils.utils import pdf_loader,pdf_2_chunks,pinecone_init,load_chunks_2_pinecone,delete_pinecone_index,pinecone_status,query_pinecone

from key_storage import key_dict
print(list(key_dict.keys()))

%reload_ext autoreload
%autoreload 2

['OPENAI_API_KEY', 'PINECONE_API_KEY', 'PINECONE_API_ENV', 'PROXYCURL_API_KEY', 'SERPAPI_API_KEY']


/Users/venetispallikaras/opt/anaconda3/envs/nlp_pip/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# token_counts = [tiktoken_len(doc.page_content) for doc in documents]
# print(f"""Min: {min(token_counts)}
# Avg: {int(sum(token_counts) / len(token_counts))}
# Max: {max(token_counts)}""")

In [3]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # set style and color palette for the plot
# sns.set_style("whitegrid")
# sns.set_palette("muted")

# # create histogram
# plt.figure(figsize=(12, 6))
# sns.histplot(token_counts, kde=False, bins=50)

# # customize the plot info
# plt.title("Token Counts Histogram")
# plt.xlabel("Token Count")
# plt.ylabel("Frequency")

# plt.show()


In [4]:
"""Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.

11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.

11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.

11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: 28-pl

QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: 0-pl
Content: And we won’t stop. 

We have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. 

Let’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  

Let’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  

We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. 

I recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. 

They were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. 

Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: 24-pl
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  

To all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. 

And I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. 

Tonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  

America will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  

These steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. 

But I want you to know that we are going to be okay.
Source: 5-pl
Content: More support for patients and families. 

To get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. 

It’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  

ARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. 

A unity agenda for the nation. 

We can do this. 

My fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. 

In this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. 

We have fought for freedom, expanded liberty, defeated totalitarianism and terror. 

And built the strongest, freest, and most prosperous nation the world has ever known. 

Now is the hour. 

Our moment of responsibility. 

Our test of resolve and conscience, of history itself. 

It is in this moment that our character is formed. Our purpose is found. Our future is forged. 

Well I know this nation.
Source: 34-pl
=========
FINAL ANSWER: The president did not mention Michael Jackson.
SOURCES:

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""

'Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement

In [5]:
# from langchain.prompts import PromptTemplate

# template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
# If you don't know the answer, just say that you don't know. Don't try to make up an answer.
# ALWAYS return a "SOURCES" part in your answer.

# QUESTION: Which state/country's law governs the interpretation of the contract?
# =========
# Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
# Source: 28-pl
# Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
# Source: 30-pl
# Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
# Source: 4-pl
# =========
# FINAL ANSWER: This Agreement is governed by English law.
# SOURCES: 28-pl

# QUESTION: What did the president say about Michael Jackson?
# =========
# Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
# Source: 0-pl
# Content: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
# Source: 24-pl
# Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
# Source: 5-pl
# Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
# Source: 34-pl
# =========
# FINAL ANSWER: The president did not mention Michael Jackson.
# SOURCES:

# QUESTION: {question}
# =========
# {summaries}
# =========
# FINAL ANSWER:"""
# PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# EXAMPLE_PROMPT = PromptTemplate(
#     template="Content: {page_content}\nSource: {source}",
#     input_variables=["page_content", "source"],
# )

In [6]:
"""Write a comprehensive summary of this financial report. 

Divide it into,
1. Financial highlights
2. Analysis of Financial results
3. Trends
4. Risks
5. Conclusion

{text}

SUMMARY:"""

'Write a comprehensive summary of this financial report. \n\nDivide it into,\n1. Financial highlights\n2. Analysis of Financial results\n3. Trends\n4. Risks\n5. Conclusion\n\n{text}\n\nSUMMARY:'

In [7]:
#tokens viz #todo
#todotodo

In [8]:
documents = pdf_loader(path="files//",loader="unstructured") #todo try a different file loader one page per document?
documents 

No documents: 2


[Document(page_content='JPMORGAN CHASE BANK, NATIONAL ASSOCIATION (a wholly-owned subsidiary of JPMorgan Chase & Co.)\n\nCONSOLIDATED FINANCIAL STATEMENTS For the three years ended December 31, 2022\n\nJPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements\n\n2\n\nConsolidated statements of income JPMorgan Chase Bank, National Association (a wholly-owned subsidiary of JPMorgan Chase & Co.)\n\nYear ended December 31, (in millions)\n\n2022\n\n2021\n\n2020\n\nRevenue\n\nInvestment banking fees\n\nPrincipal transactions\n\nLending- and deposit-related fees\n\nAsset management, administration and commissions\n\nInvestment securities gains/(losses)\n\nMortgage fees and related income\n\nCard income\n\nOther income\n\nNoninterest revenue\n\nInterest income\n\nInterest expense\n\nNet interest income\n\nTotal net revenue\n\nProvision for credit losses\n\nNoninterest expense\n\nCompensation expense\n\nOccupancy expense\n\nTechnology, communications and equipment expense

In [9]:
len(documents)

2

In [10]:
chunks = pdf_2_chunks(documents,chunk_size=512,chunk_overlap=51)
print(len(chunks))
chunks

20


[Document(page_content='JPMORGAN CHASE BANK, NATIONAL ASSOCIATION (a wholly-owned subsidiary of JPMorgan Chase & Co.)\n\nCONSOLIDATED FINANCIAL STATEMENTS For the three years ended December 31, 2022\n\nJPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements\n\n2\n\nConsolidated statements of income JPMorgan Chase Bank, National Association (a wholly-owned subsidiary of JPMorgan Chase & Co.)\n\nYear ended December 31, (in millions)\n\n2022\n\n2021\n\n2020\n\nRevenue\n\nInvestment banking fees\n\nPrincipal transactions\n\nLending- and deposit-related fees\n\nAsset management, administration and commissions\n\nInvestment securities gains/(losses)\n\nMortgage fees and related income\n\nCard income\n\nOther income\n\nNoninterest revenue\n\nInterest income\n\nInterest expense\n\nNet interest income\n\nTotal net revenue\n\nProvision for credit losses\n\nNoninterest expense\n\nCompensation expense\n\nOccupancy expense\n\nTechnology, communications and equipment expense

In [75]:
documents = pdf_loader(path="files//",loader="pypdf") #todo try a different file loader one page per document?
documents #one document per page

No documents: 22


[Document(page_content='              JPMORGAN CHASE BANK, NATIONAL ASSOCIATION (a wholly-owned subsidiary of JPMorgan Chase & Co.)            CONSOLIDATED FINANCIAL STATEMENTS For the three years ended December 31, 2022  ', metadata={'source': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf', 'page': 0}),
 Document(page_content='JPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements 2  ', metadata={'source': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf', 'page': 1}),
 Document(page_content='JPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements 3  Consolidated statements of income  JPMorgan Chase Bank, National Association (a wholly-owned subsidiary of JPMorgan Chase & Co.) Year ended December 31, (in millions) 2022 2021 2020 Revenue    Investment banking fees $ 2,955 $ 5,232 $ 3,527 Principal transactions 17,895 13,677 14,784 Lending- and deposit-re

In [76]:
chunks = pdf_2_chunks(documents,chunk_size=512,chunk_overlap=51)
print(len(chunks))
chunks

39


[Document(page_content='JPMORGAN CHASE BANK, NATIONAL ASSOCIATION (a wholly-owned subsidiary of JPMorgan Chase & Co.)            CONSOLIDATED FINANCIAL STATEMENTS For the three years ended December 31, 2022', metadata={'source': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf', 'page': 0}),
 Document(page_content='JPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements 2', metadata={'source': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf', 'page': 1}),
 Document(page_content='JPMorgan Chase Bank, National Association/2022 Consolidated Financial Statements 3  Consolidated statements of income  JPMorgan Chase Bank, National Association (a wholly-owned subsidiary of JPMorgan Chase & Co.) Year ended December 31, (in millions) 2022 2021 2020 Revenue    Investment banking fees $ 2,955 $ 5,232 $ 3,527 Principal transactions 17,895 13,677 14,784 Lending- and deposit-related fees 7,095 7

20


In [2]:
index = pinecone_init(key_dict["PINECONE_API_KEY"],key_dict["PINECONE_API_ENV"],index_name="langchain-pdf-qna",distance_metric="cosine",embed_dimension=1536)
index

Index status: 

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39}},
 'total_vector_count': 39}


In [79]:
#delete_pinecone_index(index_name="langchain-pdf-qna")

In [12]:
pinecone_status(index)

Index status: 

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39}},
 'total_vector_count': 39}


In [81]:
load_chunks_2_pinecone(chunks,index,key_dict["OPENAI_API_KEY"],embeddings_model_name='text-embedding-ada-002',batch_limit=100)

  0%|          | 0/39 [00:00<?, ?it/s]

In [82]:
pinecone_status(index)

Index status: 

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 39}},
 'total_vector_count': 39}


In [3]:
query = """Write a comprehensive summary of this financial report. 

Divide it into,
1. Financial highlights
2. Analysis of Financial results
3. Trends
4. Risks
5. Conclusion

{text}

SUMMARY:"""

In [4]:
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=3,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'Write a comprehensive summary of this financial report. \n\nDivide it into,\n1. Financial highlights\n2. Analysis of Financial results\n3. Trends\n4. Risks\n5. Conclusion\n\n{text}\n\nSUMMARY:',
 'answer': "1. Financial highlights:\n- Revenue was $985mm, up $1.9B YoY\n- Net interest income was $1.7B, compared with a loss of $536mm in the prior year, due to the impact of higher rates\n- Noninterest revenue was a loss of $755mm, compared with a loss of $345mm in the prior year, and included net investment securities losses of $868mm\n- Noninterest expense of $160mm, down $24mm YoY\n- Credit costs of $370mm, reflecting a net reserve build, driven by single name exposures\n- 1Q23 net income of $12.6B and EPS of $4.10\n- Managed revenue of $39.3B, included net investment securities losses of $868mm\n- Expense of $20.1B and managed overhead ratio of 51%\n- Common dividend of $3.0B or $1.00 per share\n- $1.9B of common stock net repurchases\n\n2. Analysis of Financial results:\n

In [5]:
query = """what is total net revenue of jpmorgan chase bank in 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is total net revenue of jpmorgan chase bank in 2021',
 'answer': 'The total net revenue of JPMorgan Chase Bank in 2021 was $104,205 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [6]:
query = """what is total net revenue of jpmorgan chase bank in year 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is total net revenue of jpmorgan chase bank in year 2021',
 'answer': 'The total net revenue of JPMorgan Chase Bank in 2021 was $104,205 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [50]:
query = """what is the total net revenue of jpmorgan chase bank in year 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is the total net revenue of jpmorgan chase bank in year 2021',
 'answer': 'The total net revenue of JPMorgan Chase Bank in year 2021 was $118,977 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [48]:
query = """what is total net income of jpmorgan chase bank in year 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is total net income of jpmorgan chase bank in year 2021',
 'answer': 'The total net income of JPMorgan Chase Bank in the year 2021 was $38,055 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [7]:
query = """what is total net revenue of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is total net revenue of jpmorgan chase bank in year 2022',
 'answer': 'The total net revenue of JPMorgan Chase Bank in 2022 was $118,977 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [52]:
query = """what is the total net revenue of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is the total net revenue of jpmorgan chase bank in year 2022',
 'answer': 'There is no information provided about the total net revenue of JPMorgan Chase Bank in the year 2022. \n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [55]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': 'There is no information provided about the total net revenue of JPMorgan Chase Bank in the year 2022. \n',
 'sources': 'N/A'}

#explanation about different chains 

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [83]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="refine")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': "The original answer remains the same: The Total net revenue of JPMorgan Chase Bank in year 2022 is $118,977 million. However, it should be noted that the provided source only includes information on the bank's consolidated balance sheet and does not explicitly state the total net revenue for the year 2022. Further information may be needed to determine the exact total net revenue for JPMorgan Chase Bank in 2022.",
 'sources': ''}

In [84]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="stuff")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 8f41cb4501ebdeec904ed9aa9d25c60f in your message.).


{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': 'The total net revenue of JPMorgan Chase Bank in year 2022 was $118,977 million.\n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

In [85]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_rerank")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': 'The Total net revenue of JPMorgan Chase Bank in year 2022 is $118,977 million.',
 'sources': ''}

In [86]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_reduce")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': 'There is no relevant text to answer the question about the "Total net revenue" of JPMorgan Chase Bank in year 2022. \n',
 'sources': 'files/JPMorgan_Chase_Bank_Annual_Consolidated_Financial_Statements_onlybalancesheet.pdf'}

## map_rerank

In [87]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_rerank")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': 'The Total net revenue of JPMorgan Chase Bank in year 2022 is $118,977 million.',
 'sources': ''}

In [88]:
query = """what is the total net revenue of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=10,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_rerank")

{'question': 'what is the total net revenue of jpmorgan chase bank in year 2022',
 'answer': '118,977 million',
 'sources': ''}

In [89]:
query = """what is the total net revenue of jpmorgan chase bank in year 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_rerank")

{'question': 'what is the total net revenue of jpmorgan chase bank in year 2021',
 'answer': '104,205 million',
 'sources': ''}

In [90]:
query = """what is the total net revenue of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="map_rerank")

{'question': 'what is the total net revenue of jpmorgan chase bank in year 2022',
 'answer': '118,977 million',
 'sources': ''}

## refine

In [91]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2022"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="refine")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2022',
 'answer': "The original answer remains the same: The Total net revenue of JPMorgan Chase Bank in year 2022 is $118,977 million. However, it should be noted that the provided source only includes information on the bank's consolidated statements of comprehensive income and does not explicitly state the total net revenue for the year 2022. Further information may be needed to provide a more accurate answer.",
 'sources': ''}

In [92]:
query = """what is the "Total net revenue" of jpmorgan chase bank in year 2021"""
query_pinecone(index_name="langchain-pdf-qna",embed_model_api_key=key_dict["OPENAI_API_KEY"],query=query,topk=5,embed_model_name='text-embedding-ada-002',qna_model="gpt-3.5-turbo",temperature=0.0,chain_type="refine")

{'question': 'what is the "Total net revenue" of jpmorgan chase bank in year 2021',
 'answer': 'The original answer remains valid as the new context does not provide a specific figure for total net revenue. The provided source only includes information on the bank\'s balance sheet and not its revenue specifically. Therefore, the "Total net revenue" of JPMorgan Chase Bank in year 2021 is still unknown.',
 'sources': ''}